In [2]:
import streamlit as st 
from scrapper import fk_scrapper_functions as sc
from scrapper import flipkart_json_scrapper_with_all_specifications as fk_scrapper
import time
import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
competitor_data=pd.read_pickle('competitor_data.pkl')

d:\personal_git\data science\portfolio projects\Competition analysis\.venv\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
competitor_data=pd.read_pickle('competitor_data.pkl')
st.write(competitor_data['all_specs'][0].get('Model Name'))
competitor_data['brand']=competitor_data['title'].str.split(" ").str[0]
brand_level_data=competitor_data.copy()
st.dataframe(brand_level_data)
for index, row in brand_level_data.iterrows():
    specs = row['all_specs']
    for key, value in specs.items():
        if key not in brand_level_data.columns:
            brand_level_data[key] = None
        brand_level_data.at[index, key] = value
most_important_feature={}
feature_name=[]
feature_count=[]
for column in brand_level_data.columns:
    feature_name.append(column)
    feature_count.append(brand_level_data[column].count())
most_important_feature['feature_name']=feature_name
most_important_feature['feature_count']=feature_count
important_features=pd.DataFrame(most_important_feature)
important_features['feautre_present_in_%_of_products']=important_features['feature_count']*100/important_features['feature_count'].max()
filtered_important_features=important_features[important_features['feautre_present_in_%_of_products'] >=10]['feature_name'].to_list()
final_scrapped_data=brand_level_data[filtered_important_features]


2024-07-01 11:55:37.974 Serialization of dataframe to Arrow table was unsuccessful due to: ("Could not convert '' with type str: tried to convert to double", 'Conversion failed for column rating with type object'). Applying automatic fixes for column types to make the dataframe Arrow-compatible.


In [127]:
final_scrapped_data[final_scrapped_data['Headphone Type'].str.lower().str.contains('black')]

,fsn,title,rating,ratings_count,reviews_count,special_price,final_selling_price,mrp,flipkart_assured,Seller Name,highlights,paymentOptions,description,productDescription,specifications,fk_other_seller_info,reviews,productImagesCount,productVideosCount,image_link,all_specs,brand,Model Name,Color,Headphone Type,Inline Remote,Sales Package,Connectivity,Headphone Design,Accessories Included,Compatible Devices,Net Quantity,Sweat Proof,Foldable/Collapsible,Deep Bass,Water Resistant,Monaural,Open/Closed Back,Carrier Frequency,Magnet Type,Connector Plating,Connector Size,Controls,Other Features,Number of Pins,Boom Microphone,With Microphone,Sensitivity,Impedance,Signal to Noise Ratio,Modulation,Maximum Power Input,Minimum Frequency Response,Maximum Frequency Response,Noise Reduction,Other Sound Features,Audio Codec,Wireless Type,Wireless Range,Bluetooth Profiles,Bluetooth Version,Bluetooth Range,Headphone Power Source,Headphone Power Requirement,Headphone Battery,Power Supply,Battery Output,Battery Type,Battery Life,Battery Capacity,Charging Time,Play Time,Standby Time,Other Power Features,Warranty Summary,Warranty Service Type,Covered in Warranty,Not Covered in Warranty,Designed For,Series,System Requirements,Circumaural/Supraaural,Driver Type,Indicators,Theme,Technology Used,Headphone Driver Units,Cord Type,Transmitter Power Source,Width,Height,Depth,Weight,Cord Length,Other Dimensions,Domestic Warranty


In [126]:
final_scrapped_data[['Color']].value_counts()

Color          
White              279
Black              272
Blue                78
Green               40
Grey                18
                  ... 
Black Neon           1
Black Or Silver      1
Silver, Green        1
Black and Red        1
Active Teal          1
Name: count, Length: 292, dtype: int64

In [72]:
final_scrapped_data['final_selling_price']=final_scrapped_data['final_selling_price'].astype(float)

C:\Users\kashy\AppData\Local\Temp\ipykernel_7200\3423002352.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_scrapped_data['final_selling_price']=final_scrapped_data['final_selling_price'].astype(float)


In [70]:
min_price=final_scrapped_data['final_selling_price'].min()
max_price=final_scrapped_data['final_selling_price'].max()

In [76]:
final_scrapped_data=final_scrapped_data[(final_scrapped_data['final_selling_price']>min_price)&(final_scrapped_data['final_selling_price']<max_price)]

In [91]:
import analysis_function as f

In [95]:
final_scrapped_data['ratings_count']=final_scrapped_data['ratings_count'].apply(lambda x: x if f.is_integer(x) else None)
final_scrapped_data['reviews_count']=final_scrapped_data['reviews_count'].apply(lambda x: x if f.is_integer(x) else None)
final_scrapped_data[(final_scrapped_data['ratings_count']>0) | ((final_scrapped_data['reviews_count']>0))]

C:\Users\kashy\AppData\Local\Temp\ipykernel_7200\4159276515.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_scrapped_data['ratings_count']=final_scrapped_data['ratings_count'].apply(lambda x: x if f.is_integer(x) else None)
C:\Users\kashy\AppData\Local\Temp\ipykernel_7200\4159276515.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_scrapped_data['reviews_count']=final_scrapped_data['reviews_count'].apply(lambda x: x if f.is_integer(x) else None)


In [ ]:
final_scrapped_data[(final_scrapped_data['ratings_count']>0) | ((final_scrapped_data['reviews_count']>0))]

In [30]:
options_to_select = []
for column in filtered_important_features:
    if column not in competitor_data.columns.to_list():
        options_to_select.append(column)

In [ ]:
Brand_dominance=pd.DataFrame(final_scrapped_data['brand'].value_counts())
Brand_dominance.reset_index(inplace=True)
Brand_dominance['Market_share']=Brand_dominance['count']*100/Brand_dominance['count'].sum()
Brand_dominance.loc[Brand_dominance['count']<5,'Brand_type'] = 'others'
Brand_dominance['Brand_type'].fillna(Brand_dominance['brand'],inplace=True)
Brand_dominance.groupby('Brand_type')['count'].sum()

In [100]:
other=Brand_dominance[Brand_dominance['count']<5]
other_brands=other.groupby('count').agg({'brand': lambda x: list(x)})
other_brands.reset_index(inplace=True)
other_brands=other_brands.rename({'count':'No of products'},axis=1)